In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from typing import Dict

import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import csr_matrix

import celeri

# Read in data files and do basic processing

In [ ]:
# Japan example
command_file_name = "../data/command/japan_command.json"

# Western North America example
# command_file_name = "../data/command/western_north_america_command.json"

In [ ]:
command = celeri.get_command(command_file_name)
celeri.create_output_folder(command)
logger = celeri.get_logger(command)
segment, block, meshes, station, mogi, sar = celeri.read_data(command)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

# Get elastic operators and TDE smoothing operators

In [ ]:
# Calculate Okada partials for all segments
celeri.get_elastic_operators_okada(operators, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)

# Calculate non-elastic operators

In [ ]:
operators.rotation_to_velocities = celeri.get_rotation_to_velocities_partials(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_partials(station)
assembly, operators.block_motion_constraints = celeri.get_block_motion_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.get_slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_rotation_to_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_index = celeri.get_block_strain_rate_to_velocities_partials(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_to_velocities_partials(mogi, station, command)
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
celeri.get_tde_slip_rate_constraints(meshes, operators)

# Plot input data

In [ ]:
celeri.plot_input_summary(segment, station, block, meshes, mogi, sar, lon_range=command.lon_range, lat_range=command.lat_range, quiver_scale=command.quiver_scale)

# Calculate col_norms and H for each mesh here.

In [ ]:
index = celeri.get_index(assembly, station, block, meshes)

# Data and data weighting vector
weighting_vector = celeri.get_weighting_vector(command, station, meshes, index)
data_vector = celeri.get_data_vector(assembly, index)

# Apply data weighting
data_vector = data_vector * np.sqrt(weighting_vector)

from celeri.hmatrix import build_hmatrix_from_mesh_tdes

# Cast all block submatrices to sparse
sparse_block_motion_okada_faults = csr_matrix(operators.rotation_to_velocities[index.station_row_keep_index, :] - operators.rotation_to_slip_rate_to_okada_to_velocities[index.station_row_keep_index, :])
sparse_block_motion_constraints = csr_matrix(operators.block_motion_constraints)
sparse_block_slip_rate_constraints = csr_matrix(operators.slip_rate_constraints)

# Calculate column normalization vector for blocks
operator_block_only = celeri.get_full_dense_operator_block_only(operators, index)
weighting_vector_block_only = weighting_vector[0:operator_block_only.shape[0]][:, None]
col_norms = np.linalg.norm(operator_block_only * np.sqrt(weighting_vector_block_only), axis=0)


# Create lists for all TDE matrices per mesh
H = []
for i in range(len(meshes)):
    # Get full TDE to velocity matrix for current mesh
    tde_to_velocities = celeri.get_elastic_operator_single_mesh(meshes, station, command, i)
    
    # H-matrix representation
    H.append(build_hmatrix_from_mesh_tdes(
        meshes[i],
        station,
        -tde_to_velocities,
        command.h_matrix_tol,
        command.h_matrix_min_separation,
        command.h_matrix_min_pts_per_box,
    ))

    logger.info(f"mesh {i} ({meshes[i].name}) H-matrix compression ratio: {H[i].report_compression_ratio():0.4f}")

    # Case smoothing matrices and tde slip rate constraints to sparse
    smoothing_keep_index = celeri.get_keep_index_12(operators.smoothing_matrix[i].shape[0])
    operators.smoothing_matrix[i] = csr_matrix(operators.smoothing_matrix[i][smoothing_keep_index, :][:, smoothing_keep_index])    
    operators.tde_slip_rate_constraints[i] = csr_matrix(operators.tde_slip_rate_constraints[i])

    # Eliminate unused columns and rows of TDE to velocity matrix
    tde_to_velocities = np.delete(tde_to_velocities, np.arange(2, tde_to_velocities.shape[0], 3), axis=0)
    tde_to_velocities = np.delete(tde_to_velocities, np.arange(2, tde_to_velocities.shape[1], 3), axis=1)

    # Calculate column normalization vector current TDE mesh
    weighting_vector_no_zero_rows = celeri.get_weighting_vector_single_mesh_for_col_norms(command, station, meshes, index, i)
    current_tde_mesh_columns_full_no_zero_rows = np.vstack((-tde_to_velocities, operators.smoothing_matrix[i].toarray(), operators.tde_slip_rate_constraints[i].toarray())) * np.sqrt(weighting_vector_no_zero_rows[:, None])

    # Concatenate everthing we need for col_norms
    col_norms_current_tde_mesh = np.linalg.norm(current_tde_mesh_columns_full_no_zero_rows, axis=0)
    col_norms = np.hstack((col_norms, col_norms_current_tde_mesh))


In [ ]:
# Package parameters that matvec and rmatvec need for the iterative solve
h_matrix_solve_parameters = (index,
                             meshes,
                             H,
                             operators,
                             weighting_vector,
                             col_norms,
                             sparse_block_motion_okada_faults,
                             sparse_block_motion_constraints,
                             sparse_block_slip_rate_constraints,
                             )

# Instantiate the scipy the linear operator for the iterative solver to use
operator_hmatrix = scipy.sparse.linalg.LinearOperator((index.n_operator_rows, index.n_operator_cols), matvec=celeri.matvec_wrapper(h_matrix_solve_parameters), rmatvec=celeri.rmatvec_wrapper(h_matrix_solve_parameters))

# Solve the linear system
sparse_hmatrix_solution = scipy.sparse.linalg.lsmr(operator_hmatrix, data_vector, atol=command.atol, btol=command.btol)

# Correct the solution for the col_norms preconditioning.
sparse_hmatrix_state_vector = sparse_hmatrix_solution[0] / col_norms


In [ ]:
estimation = addict.Dict()
estimation.data_vector = data_vector
estimation.weighting_vector = weighting_vector
estimation.operator = operator_hmatrix
estimation.state_vector = sparse_hmatrix_state_vector
celeri.post_process_estimation_hmatrix(estimation, operators, meshes, H, station, index, col_norms, h_matrix_solve_parameters)
celeri.write_output(command, estimation, station, segment, block, meshes)
celeri.plot_estimation_summary(segment, station, meshes, estimation, lon_range=command.lon_range, lat_range=command.lat_range, quiver_scale=command.quiver_scale)


In [ ]:
residual_velocity_vector = np.concatenate((estimation.east_vel_residual.values, estimation.north_vel_residual.values))
mean_average_error = np.mean(np.abs(residual_velocity_vector))
mean_squared_error = np.sum(residual_velocity_vector**2.0) / residual_velocity_vector.size

# Create histogram of residual velocities
plt.figure()
plt.hist(residual_velocity_vector, 50)
plt.xlabel("residual velocity (mm/yr)")
plt.ylabel("N")
plt.title(f"mae = {mean_average_error:.2f} (mm/yr), mse = {mean_squared_error:.2f} (mm/yr)^2")
plt.show()

print(type(estimation.east_vel_residual))
print(type(estimation.east_vel_residual.values))

